In [ ]:
##########################################################################################
#                                                                                        #
#  888888b.  8888888 .d88888b.   .d8888b.     .d8888b.   .d8888b.      d8888      d8888  #
#  888  "88b   888  d88P" "Y88b d88P  Y88b   d88P  Y88b d88P  Y88b    d8P888     d8P888  #
#  888  .88P   888  888     888 Y88b.        888        888          d8P 888    d8P 888  #
#  8888888K.   888  888     888  "Y888b.     888d888b.  888d888b.   d8P  888   d8P  888  #
#  888  "Y88b  888  888     888     "Y88b.   888P "Y88b 888P "Y88b d88   888  d88   888  #
#  888    888  888  888     888       "888   888    888 888    888 8888888888 8888888888 #
#  888   d88P  888  Y88b. .d88P Y88b  d88P   Y88b  d88P Y88b  d88P       888        888  #
#  8888888P" 8888888 "Y88888P"   "Y8888P"     "Y8888P"   "Y8888P"        888        888  #
#                                                                                        # 
##########################################################################################
#
# Wrangle CDC Anonymized Patient-Level COVID data
#
##########################################################################################

In [18]:
##################################################################################################################
#
# YouDo:
#    1) Make a copy of this notebook with your name as a prefix:  
#       YourName_BIOS6644_XML_RareDiseases.ipynb
#    2) Do all work in this new notebook.
#    3) Submit completed work via GitHub
#
##################################################################################################################


In [46]:
# Review: Python Dictionaries
# empty dictionary

d = dict() 
print(d)

# Dictionaries make 1-to-1 maps from "keys" to "values"
# keys can be anything that's "hashable" (strings, numbers, dicts, ...)
# Values can be any Python type (including custom-made)

name={'firstname':'James', 'surname':'King'}

d['name'] = name 
d['age'] = 'Younger than Greg' 
d['Favorite number'] = 3 
print(d)


{}
{'name': {'firstname': 'James', 'surname': 'King'}, 'age': 'Younger than Greg', 'Favorite number': 3}


# Setup Local Environment
- Ensure you have pandas and requests installed (via pip, conda, etc.)


## Get Data
>120 million rows (!)

https://data.cdc.gov/Case-Surveillance/COVID-19-Case-Surveillance-Public-Use-Data-with-Ge/n8mc-b4w4/about_data

In [6]:
# CDC provides an Application Programming Interface (API) to access this data
# Clicking through their help files leads to here:
# https://dev.socrata.com/docs/paging
# which shows how to choose the number of records you're downloading

# This data set has > 100 million records and spans lots of GB, so you want to do your EDA on
# a sample

# adapted from here: https://www.sitepoint.com/python-fetching-data-http-api/

import pandas as pd
import requests

def fetch_records(num=1000, offset=0, single_state=None):
    # Fetches the most recent num records
    if single_state is not None:
        stclause = f'res_state={single_state}&'
    else:
        stclause = ''
    base_url = f'https://data.cdc.gov/resource/n8mc-b4w4.json?{stclause}&$limit={num}&$order=case_month DESC&$offset={offset}'
    response = requests.get(base_url)
    print(response)
    return response.json()



In [19]:
# "top" of the data is hot garbage
# Offset to get to real records

#results = fetch_records(3,offset=0)
results = fetch_records(3,offset=20,single_state="WY")

<Response [200]>


In [21]:
#type(results), len(results), type(results[0]), results
pd.DataFrame(results)

,case_month,res_state,state_fips_code,res_county,county_fips_code,age_group,sex,race,ethnicity,case_positive_specimen,process,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn,case_onset_interval
0,2024-02,WY,56,PARK,56029,65+ years,NA,NA,NA,0.0,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,NaN
1,2024-02,WY,56,PARK,56029,65+ years,NA,NA,NA,0.0,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,NaN
2,2024-02,WY,56,PARK,56029,65+ years,NA,NA,NA,NaN,Missing,Missing,Laboratory-confirmed case,Symptomatic,No,Missing,NA,0.0


In [27]:

df=pd.DataFrame(fetch_records(num=3000, offset=0, single_state="CO"))

<Response [200]>


In [28]:
df

,case_month,res_state,state_fips_code,res_county,county_fips_code,age_group,sex,race,ethnicity,process,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn,underlying_conditions_yn
0,2024-02,CO,08,NA,NA,50 to 64 years,Female,White,NA,Missing,Missing,Probable Case,Missing,Missing,Missing,Missing,NaN
1,2024-02,CO,08,NA,NA,50 to 64 years,Female,White,NA,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,NaN
2,2024-02,CO,08,NA,NA,50 to 64 years,Female,White,NA,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,NA,NaN
3,2024-02,CO,08,NA,NA,50 to 64 years,Female,White,NA,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,NA,NaN
4,2024-02,CO,08,NA,NA,50 to 64 years,Female,White,NA,Missing,Missing,Laboratory-confirmed case,Missing,No,Missing,Missing,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2024-02,CO,08,LARIMER,08069,18 to 49 years,Female,Missing,Hispanic/Latino,Missing,Missing,Probable Case,Missing,Missing,Missing,Missing,NaN
2996,2024-02,CO,08,LARIMER,08069,18 to 49 years,Female,Missing,Hispanic/Latino,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,NaN
2997,2024-02,CO,08,LARIMER,08069,18 to 49 years,Female,Missing,Hispanic/Latino,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing,NaN
2998,2024-02,CO,08,LARIMER,08069,18 to 49 years,Female,Missing,Hispanic/Latino,Missing,Missing,Probable Case,Missing,Missing,Missing,Missing,NaN


In [29]:
chunks = []
for chunk in range(10):
    thisdf=pd.DataFrame(fetch_records(num=10, offset=10*chunk, single_state="CO"))
    chunks.append(thisdf)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [31]:
pd.concat(chunks)

,case_month,res_state,state_fips_code,res_county,county_fips_code,age_group,sex,race,ethnicity,process,exposure_yn,current_status,symptom_status,hosp_yn,icu_yn,death_yn
0,2024-02,CO,08,NA,NA,50 to 64 years,Female,White,NA,Missing,Missing,Probable Case,Missing,Missing,Missing,Missing
1,2024-02,CO,08,NA,NA,50 to 64 years,Female,White,NA,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing
2,2024-02,CO,08,NA,NA,50 to 64 years,Female,White,NA,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,NA
3,2024-02,CO,08,NA,NA,50 to 64 years,Female,White,NA,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,NA
4,2024-02,CO,08,NA,NA,50 to 64 years,Female,White,NA,Missing,Missing,Laboratory-confirmed case,Missing,No,Missing,Missing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,2024-02,CO,08,NA,NA,65+ years,Female,White,Non-Hispanic/Latino,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Missing
6,2024-02,CO,08,NA,NA,65+ years,Female,White,Non-Hispanic/Latino,Missing,Missing,Probable Case,Missing,Missing,Missing,Missing
7,2024-02,CO,08,NA,NA,65+ years,Female,White,Non-Hispanic/Latino,Missing,Missing,Probable Case,Missing,Missing,Missing,Missing
8,2024-02,CO,08,NA,NA,65+ years,Female,White,Non-Hispanic/Latino,Missing,Missing,Probable Case,Missing,No,Missing,NA


In [24]:
# All states represented?
df.res_state.value_counts()

res_state
CO    3000
Name: count, dtype: int64

In [12]:
codf = df.loc[df.res_state=="CO"]
codf.res_county.value_counts()

res_county
LARIMER      486
NA           457
EL PASO      449
DENVER       412
ARAPAHOE     392
WELD         268
ADAMS        220
JEFFERSON    192
DOUGLAS       76
BOULDER       48
Name: count, dtype: int64

# P.S.  "Indexing" dictionaries

Be careful!  The keys on a dictionary are not guaranteed to be in the same order every time you print them.

In [35]:
d1=results[0]

In [36]:
d1

{'case_month': '2024-02',
 'res_state': 'WY',
 'state_fips_code': '56',
 'res_county': 'PARK',
 'county_fips_code': '56029',
 'age_group': '65+ years',
 'sex': 'NA',
 'race': 'NA',
 'ethnicity': 'NA',
 'case_positive_specimen': '0.0',
 'process': 'Missing',
 'exposure_yn': 'Missing',
 'current_status': 'Laboratory-confirmed case',
 'symptom_status': 'Missing',
 'hosp_yn': 'Missing',
 'icu_yn': 'Missing',
 'death_yn': 'Missing'}

In [39]:
kk = list(d1.keys())
kk

['case_month',
 'res_state',
 'state_fips_code',
 'res_county',
 'county_fips_code',
 'age_group',
 'sex',
 'race',
 'ethnicity',
 'case_positive_specimen',
 'process',
 'exposure_yn',
 'current_status',
 'symptom_status',
 'hosp_yn',
 'icu_yn',
 'death_yn']

In [40]:
# Index the keys
kk[3]

'res_county'

In [41]:
d1[kk[3]]

'PARK'

In [44]:
d1.items()


dict_items([('case_month', '2024-02'), ('res_state', 'WY'), ('state_fips_code', '56'), ('res_county', 'PARK'), ('county_fips_code', '56029'), ('age_group', '65+ years'), ('sex', 'NA'), ('race', 'NA'), ('ethnicity', 'NA'), ('case_positive_specimen', '0.0'), ('process', 'Missing'), ('exposure_yn', 'Missing'), ('current_status', 'Laboratory-confirmed case'), ('symptom_status', 'Missing'), ('hosp_yn', 'Missing'), ('icu_yn', 'Missing'), ('death_yn', 'Missing')])